In [1]:
import numpy as np
np.seterr(all='raise')
import sys
sys.path.append('../')

from meshmaker.base import Base, Laziness
from meshmaker.model import Model
from meshmaker.mesh import Mesh
from meshmaker.pmesh import ParamMesh, MetaMesh, MetaScene
#from meshmaker.meta import Railing, Stairs
from meshmaker.tform import TForm
from meshmaker.vec3 import vec3
from meshmaker.quat import quat
from meshmaker.delaunay import triangulation
from meshmaker.planargraph import planargraph
from meshmaker.geometry import batch, slide, loop_offset, loop_normal, loop_contains, loop_split, isnear, near, loopO
from meshmaker.mgl import show, MainShader, EdgeShader, WireShader, LazyMaterials
from meshmaker.plt import *
from collections import defaultdict
from functools import partial, reduce
import json

show = partial(show, programs=[MainShader(), EdgeShader(), WireShader(color=vec3.U(0.1))], background=vec3.U(0.8))

In [ ]:
class BSP:
    """Reimplementation of: https://github.com/evanw/csg.js/blob/master/csg.js
    Holds a node in a BSP tree. A BSP tree is built from a collection of polygons
    by picking a polygon to split along. That polygon (and all other coplanar
    polygons) are added directly to that node and the other polygons are added to
    the front and/or back subtrees. This is not a leafy BSP tree since there is
    no distinction between internal and leaf nodes."""
    
    @property
    def W(self):
        return self.O.dot(self.N)
    
    def __init__(self, loops):
        self.O = None
        self.N = None
        self.L = None
        self.R = None
        self.loops = []
        self.build(loops)
        
    def cp(self):
        """Create an independent copy of this BSP instance."""
        bsp = BSP([])
        bsp.O = self.O.cp()
        bsp.N = self.N.cp()
        bsp.L = self.L.cp() if self.L is not None else None
        bsp.R = self.R.cp() if self.R is not None else None
        bsp.loops = [[p.cp() for p in l] for l in self.loops]
        return bsp
    
    @classmethod
    def from_mesh(self, mesh):
        """Create a BSP from a brep mesh"""
        loops = [[mesh.vertices[v].cp() for v in face] for f, face in mesh]
        return BSP(loops)
    
    @classmethod
    def from_plane(self, O, N, r=1000):
        """Create a BSP from a division plane"""
        loop = vec3.O().ring(r, 4)
        O.trnps(quat.toxy(N).fp().rot(loop))
        bsp = BSP([loop])
        bsp.O = O
        bsp.N = N
        return bsp

    def split_loop(self, loop, coL, coR, L, R, eps=0.000001):
        """Split `polygon` by this plane if needed, then put the polygon or polygon
        fragments in the appropriate lists. Coplanar polygons go into either
        `coL` or `coR` depending on their orientation with respect to this plane.
        Polygons in front or in back of this plane go into either `front` or `back`.
        """
        COPLANAR, FRONT, BACK, SPANNING = 0, 1, 2, 3
        ltype = 0
        types = []
        for p in loop:
            t = self.N.dot(p) - self.W
            ptype = (BACK if t < -eps else (FRONT if t > eps else COPLANAR))
            ltype |= ptype
            types.append(ptype)
        if ltype == COPLANAR:
            if self.N.dot(loop_normal(loop)) > 0:
                coL.append(loop)
            else:
                coR.append(loop)
        elif ltype == FRONT:
            L.append(loop)
        elif ltype == BACK:
            R.append(loop)
        elif ltype == SPANNING:
            l, r = [], []
            n = len(loop)
            for i in range(n):
                j = (i + 1) % n
                ti, tj = types[i], types[j]
                vi, vj =  loop[i],  loop[j]
                if ti != BACK:
                    l.append(vi)
                if ti != FRONT:
                    r.append(vi.cp() if ti != BACK else vi)
                if (ti | tj) == SPANNING:
                    t = (self.W - self.N.dot(vi)) / self.N.dot(vj - vi)
                    v = vi.lerp(vj, t)
                    l.append(v)
                    r.append(v.cp())
            if len(l) >= 3:
                L.append(l)
            if len(r) >= 3:
                R.append(r)
        
    def invert(self):
        """Convert solid space to empty space and empty space to solid space."""
        for loop in self.loops:
            loop.reverse()
        self.N = self.N.fp()
        if self.L:
            self.L.invert()
        if self.R:
            self.R.invert()
        self.L, self.R = self.R, self.L
        
    def clip_loops(self, loops):        
        """Recursively remove all polygons in `polygons` that are inside this BSP tree."""
        if self.N is None:
            return loops[:]
        L, R = [], []
        for loop in loops:
            self.split_loop(loop, L, R, L, R)
        if self.L:
            L = self.L.clip_loops(L)
        if self.R:
            R = self.R.clip_loops(R)
        else:
            R = []
        return L + R
    
    def clip_to(self, bsp):
        """Remove all polygons in this BSP tree that are inside the other BSP tree `bsp`."""
        self.loops = bsp.clip_loops(self.loops)
        if self.L:
            self.L.clip_to(bsp)
        if self.R:
            self.R.clip_to(bsp)
        
    def all_loops(self):
        """Return a list of all polygons in this BSP tree."""
        loops = self.loops[:]
        if self.L:
            loops.extend(self.L.all_loops())
        if self.R:
            loops.extend(self.R.all_loops())
        return loops
        
    def build(self, loops):
        """Build a BSP tree out of `polygons`. When called on an existing tree, the
        new polygons are filtered down to the bottom of the tree and become new
        nodes there. Each set of polygons is partitioned using the first polygon
        (no heuristic is used to pick a good split)."""
        if not loops:
            return
        if self.N is None:
            self.O = loops[0][0].cp()
            self.N = loop_normal(loops[0])
        L, R = [], []
        for loop in loops:
            self.split_loop(loop, self.loops, self.loops, L, R)
        if L:
            if self.L is None:
                self.L = BSP([])
            self.L.build(L)
        if R:
            if self.R is None:
                self.R = BSP([])
            self.R.build(R)

    def union(self, other):
        """Add other to self"""
        self.clip_to(other)
        other.clip_to(self)
        other.invert()
        other.clip_to(self)
        other.invert()
        self.build(other.all_loops())
        return self
    
    def difference(self, other):
        """Subtract other from self"""
        self.invert()
        self.clip_to(other)
        other.clip_to(self)
        other.invert()
        other.clip_to(self)
        other.invert()
        self.build(other.all_loops())
        self.invert()
        return self
    
    def intersect(self, other):
        """Subtract the difference of self and other from self"""
        self.invert()
        other.clip_to(self)
        other.invert()
        self.clip_to(other)
        other.clip_to(self)
        self.build(other.all_loops())
        self.invert()
        return self
    
    def split(self, O, N):
        """Split a BSP tree into two trees using a plane"""
        L, R = self.cp(), self.cp()
        L = L.difference(BSP.from_plane(O, N))
        R = R.difference(BSP.from_plane(O.cp(), N.fp()))
        return L, R
            
    def mesh(self):
        """Create a brep mesh for this BSP"""
        mesh = Mesh()
        for loop in self.all_loops():
            mesh.af(loop)
        return mesh

    @classmethod
    def brep_union(cls, *breps):
        """Find the union brep of multiple breps"""
        bsps = [cls.from_mesh(brep) for brep in breps]
        return reduce(lambda x, y: x.union(y), bsps).mesh()
        
    @classmethod
    def brep_difference(cls, a, b):
        """Find the difference of one brep from another"""
        return cls.from_mesh(a).difference(cls.from_mesh(b)).mesh()

    @classmethod
    def brep_intersect(cls, *breps):
        """Find the intersection of one brep and another"""
        bsps = [cls.from_mesh(brep) for brep in breps]
        return reduce(lambda x, y: x.intersect(y), bsps).mesh()
    
    @classmethod
    def brep_split(cls, a, O, N):
        """Split via a plane"""
        u, v = cls.from_mesh(a).split(O, N)
        return u.mesh(), v.mesh()

In [ ]:
def explode(*meshes, O=vec3.O(), d=1):
    for mesh in meshes:
        com = vec3.com(mesh.vertices)
        ((com - O) * d).trnps(mesh.vertices)
    return meshes

def compact(*meshes, d=0.1):
    for mesh in meshes:
        com = vec3.com(mesh.vertices)
        com.trnps(vec3.U(1 - d).sclps(com.fp().trnps(mesh.vertices)))
    return meshes    

In [ ]:
a = Mesh.cube_mesh()
b = Mesh.cube_mesh()
vec3.U(-0.5).trnps(a.vertices)
vec3.U( 0.5).trnps(b.vertices)
show(TForm(models=[Model(meshes={'generic_11': [a], 'generic_10': [b]})]))
#show([a, b])

In [ ]:
show(a.union(b))
show(b.union(a))
show(a.difference(b))
show(b.difference(a))
show(a.intersect(b))
show(b.intersect(a))

In [ ]:
show(BSP.brep_union(a, b))
show(BSP.brep_union(b, a))
show(BSP.brep_difference(a, b))
show(BSP.brep_difference(b, a))
show(BSP.brep_intersect(a, b))
show(BSP.brep_intersect(b, a))

In [ ]:
a = Mesh.cube_mesh()

u, v = BSP.brep_split(a, Plane(vec3.Z(0.2), vec3(0.2, 0.2, 0.8).nrm()))
u, y = BSP.brep_split(u, Plane(vec3.Z(0.6), vec3(-0.2, 0.2, 0.0).nrm()))
v, w = BSP.brep_split(v, Plane(vec3.X(0.5), vec3.X()))
v, x = BSP.brep_split(v, Plane(vec3.X(0.3), vec3.U().xy()))

show(compact(u, v, w, x, y))

In [ ]:
a = Mesh.cube_mesh(1)

a = BSP.from_brep(a)
b = BSP.from_plane(Plane(vec3.O(), vec3.Z()))
b.build(a.all_loops())

show(compact(*b.breps()))

#show([a, b])

# use CSG to design control meshes which cover 3d volume

# with quat syntax for planar operations?

In [ ]:
class PlanarOperation:
    
    def __init__(self, N, f):
        self.f = f
        self.N = N.cp()
        self.Q = quat.toxy(self.N)
        self.Q_i = quat.toxy(self.N)

    def __call__(self, *ags, **kws):
        for arg in ags:
            self.Q.rot(arg)
        result = self.f(*ags, **kws)
        Qi = self.Q.fp()
        for points in result:
            Qi.rot(points)
        for arg in ags:
            Qi.rot(arg)
        return result

In [2]:
def planar(operation):
    """Perform an operation on coplanar polygons by transforming to/from the xy-plane
    
    Args:
        operation (function): Function which returns 1+ loops given 1+ loops after
                              applying some operation in the xy-plane.
    
    """
    def wrap(*loops, **kws):
        rot = quat.toxy(loop_normal(loops[0]))
        for loop in loops:
            rot.rot(loop)
        result = operation(*loops, **kws)
        rot = rot.fp()
        for loop in loops:
            rot.rot(loop)
        for loop in result:
            rot.rot(loop)
        return result
    return wrap

In [3]:


# TODO: wtf is the correct interface of boolean loop operations... (use loop sets to support holes??)

@planar
def loop_intersection(l1, l2):
    """Find the intersection of two loops"""
    l1 = [p.quant() for p in l1]
    l2 = [p.quant() for p in l2]
    segs = list(slide(l1, 2)) + list(slide(l2, 2))
    pg = planargraph(segs=segs)
    _, iloops = pg.polygon()
    parts = []
    for loop in iloops:
        for p in loop:
            if (p.inbxy(l1, True) and p.inbxy(l2, True)):
                continue
            else:
                break
        else:
            parts.append(loop)
    return parts


@planar
def loop_difference(l1, l2):
    
    # to reduce, take 2, return 1
    # list of positives, list of negatives, list of results
    # this probably just cant be reducible... 
    
    #if others:
    #    return reduce(loop_difference, [l1, l2] + list(others))

    print(l1)
    print(l2)

    l1 = [[p.quant() for p in l] for l in l1]
    l2 = [[p.quant() for p in l] for l in l2]
    segs = [list(slide(l, 2)) for l in (l1 + l2)]
    segs = [x for y in segs for x in y]

    #l1 = [p.quant() for p in l1]
    #l2 = [p.quant() for p in l2]
    #segs = list(slide(l1, 2)) + list(slide(l2, 2))
    
    #segs = [(v, u) for u, v in segs]
    pg = planargraph(segs=segs)
    
    #f, ax = plot()
    #plot_pg(ax, pg, lw=5, col='k')
    #for u, v in segs:
    #    plot_edge(ax, u, v, lw=2, col='r')
    
    _, iloops = pg.polygon()
    parts = []
    for loop in iloops:
        for p in loop:
            if not any(p.inbxy(l, True) for l in l2):
                parts.append(loop)
                break
    return parts


@planar
def loop_union(l1, l2):
    l1 = [p.quant() for p in l1]
    l2 = [p.quant() for p in l2]
    segs = list(slide(l1, 2)) + list(slide(l2, 2))
    pg = planargraph(segs=segs)
    union, _ = pg.polygon()
    return [union]


def polygon_difference(py1, py2):
    """py2[1] is within negative space by construction"""
    
    pos = py1[0]
    neg = py1[1] + [py2[0]]
    
    if len(neg) > 1:
        neg = reduce(loop_union, neg)
        
    holes = []
    for hole in neg:
        if loop_contains(pos, hole):
            holes.append(hole)
    for hole in holes:
        neg.remove(hole)

    pos = [pos]
    if len(neg) > 0:
        pos = reduce(loop_difference, pos + neg)
        
    pys = []
    for p in pos:
        pys.append((p, [h for h in holes if loop_contains(p, h)]))

    return pys

In [ ]:
def test():
    l1, l2 = vec3(-1, -1, 0).ring(4, 4), vec3(1, 1, 0).ring(4, 4)

    segs = list(slide(l1, 2)) + list(slide(l2, 2))
    pg = planargraph(segs=segs)
    
    f, ax = plot()
    plot_pg(ax, pg, lw=6, col='g', mk='s')
    union = loop_union(l1, l2)
    for u in union:
        plot_loop(ax, u, lw=3, col='b', mk='s')

    #f, ax = plot()
    #plot_pg(ax, pg, lw=6, col='g', mk='s')
    
    #diff = loop_difference(l1, l2)
    #for d in diff:
    #    plot_loop(ax, d, lw=3, col='b', mk='s')

    f, ax = plot()
    plot_pg(ax, pg, lw=6, col='g', mk='s')
    intersection = loop_intersection(l1, l2)
    for i in intersection:
        plot_loop(ax, i, lw=3, col='b', mk='s')
    
test()

In [4]:
class Partition(Base):
    
    def __init__(self, *meshes, **kws):
        super().__init__(**kws)
        self.meshcount = 0
        self.meshes = []
        for mesh in meshes:
            self.av(mesh)
    
    def __iter__(self):
        """Yield the index, mesh pairs in the partition"""
        for i, mesh in enumerate(self.meshes):
            if mesh is not None:
                yield (i, mesh)
    
    def av(self, mesh, **kws):
        """Add new volume/vertex"""
        m = len(self.meshes)
        self.meshes.append(mesh)
        self.meshcount += 1
        return m
    
    def rv(self, v):
        """Remove volume/vertex v"""
        mesh = self.meshes[v]
        self.meshes[v] = None
        self.meshcount -= 1
        return mesh
    
    def sv(self, O, N, *vs):
        """Split vertices *vs using plane defined by O and N"""
        nvs = []
        for v in vs:
            v = self.rv(v)
            x, y = v.split(O, N)
            nvs.append(self.av(x))
            nvs.append(self.av(y))
        return nvs
    
    def graph(self):
        """Map the adjacency of volumes within the partition
        identifying the intersections of their bounding faces"""
        adjacent = defaultdict(lambda : defaultdict(lambda : {}))
        normals = {i: mesh.face_normals() for i, mesh in self}
        for i, u in self:
            for k, uF in u:
                uN = normals[i][k]
                uL = [u.vertices[x].cp() for x in uF]
                uL.reverse()
                for j, v in self:
                    if i < j:
                        continue
                    for l, vF in v:
                        vN = normals[j][l]
                        vL = [v.vertices[x].cp() for x in vF]
                        if uN.isnear(vN.fp()):
                            if isnear(uL[0].dot(uN), vL[0].dot(uN)):
                                overlap = loop_intersection(uL, vL)
                                if overlap:
                                    adjacent[i][j][k] = overlap
                                    adjacent[j][i][l] = overlap
        return adjacent

In [5]:
def Wall(supports):

    def setportals(loop, ploops):
        """Make copies of ploops which are properly coplanar with loop"""
        lN = loop_normal(loop)
        aligned = []
        for ploop in ploops:
            ploop = [p.cp() for p in ploop]
            pN = loop_normal(ploop)
            if not lN.isnear(pN):
                ploop.reverse()
            inset = [p.cp() for p in ploop]
            dN = lN * (loop[0].dot(lN) - ploop[0].dot(lN))
            dN.trnps(ploop)
            aligned.append((ploop, inset))
        return aligned

    def meta(control, faces):
        assert len(faces) == 1
        meshes = defaultdict(list)
        for f in faces:
            loop = [control.vertices[v] for v in control.faces[f]]

            N = loop_normal(loop) * 0.1
            ploops = []
            for support in supports:
                portals = support['portals']
                for portal, supp in setportals(loop, portals):
                    ploops.append(portal)
                    mesh = Mesh()
                    mesh.bridge(portal, supp)
                    meshes['generic_14'].append(mesh)

                for sloop in support['support']:
                    dbg = Mesh.cube_mesh(0.02)
                    (vec3.com(sloop) + N).trnps(dbg.vertices)
                meshes['generic_17'].append(dbg)

            mesh = Mesh()

            #mesh.af(loop)
            #for ploop in ploops:
            #    mesh.af(ploop)

            try:
                py = (
                    [p.cp().quant() for p in loop],
                    [[p.cp().quant() for p in ib] for ib in ploops])
                mesh.apy(py, 0.0001, None)
            except:
                print(((loop, ploops), ))

            meshes['generic_19'].append(mesh)

        return TForm(models=[Model(meshes=meshes)])

    return meta


    

def show_partition(partition):
    adjacent = partition.graph()

    # TODO: SEPARATE THE META REPRESENTATION TO A SUBCLASS
    # replace the meta method of interior walls
    oface = MetaMesh.textured(texture='generic_16')
    nodes = {}
    for i in adjacent:
        nodes[i] = partition.meshes[i].cp().offset(0.1).fp()
        for k, _ in nodes[i]:
            supports = [adjacent[i][j][k] for j in adjacent[i]
                                        if k in adjacent[i][j]]
            if supports:
                interfaces = [
                    {'support': support,
                     'portals': [loop_offset(loop, 0.2) for loop in support]}
                        for support in supports]
                nodes[i].meta[k] = Wall(interfaces)
                nodes[i].meta[k].__name__ = f'metawall-<{i},{k}>'
            else:
                nodes[i].meta[k] = oface
    edges = [(i, j) for i in adjacent for j in adjacent[i]]


    cube_tex = 'generic_6'
    meshes = defaultdict(list)
    for i, node in nodes.items():
        cube = Mesh.cube_mesh(0.02)
        vec3.com(node.vertices).trnps(cube.vertices)
        meshes[cube_tex].append(cube)

    def wires(tf, nodes=None, edges=None):
        wires = []
        for i, j in edges:
            wires.append((vec3.com(nodes[i].vertices),
                          vec3.com(nodes[j].vertices)))
        return wires        
    meshes[cube_tex][0].wires = partial(wires, nodes=nodes, edges=edges)

    #shell_tex = 'generic_3'
    #shell = BSP.brep_union(*filter(None, self.meshes))
    #shell = shell.offset(-0.1)
    #meshes[shell_tex].append(shell)

    mr = TForm(metas=[MetaMesh(node) for i, node in nodes.items()],
               models=[Model(meshes=meshes)])
    ms = MetaScene(mr)
    return ms



r = 4
        
#cylinder = Mesh.prism(vec3.O().ring(r / 2, 8), 3, 1.5)
#cube     = Mesh.prism(vec3.X(3).ring(r, 4), 2, 2)
#control  = BSP.brep_union(cylinder, cube)
#control = BSP.brep_difference(cube, cylinder)
#show([cylinder, cube])


#a = Mesh.cube_mesh(r)
#b = Mesh.cube_mesh(r)
#vec3.U(-r / 2).trnps(a.vertices)
#vec3.U( r / 2).trnps(b.vertices)
#control  = BSP.brep_union(a, b)


control = Mesh.cube_mesh(r)
p = Partition()
x = p.av(control)
x, y = p.sv(vec3.Z(1), vec3.Z(), x)
x, z = p.sv(vec3.Z(1), vec3.X(), x)
w, z = p.sv(vec3.Z(1), vec3.Y(), z)
y, a = p.sv(vec3.Z(1), vec3.U().xy().nrm(), y)
p.rv(w)
p.rv(y)
ms = show_partition(p)
show(ms)

Loaded texture: generic_19 (../resources/textures/generics/generic_19.png)
Loaded texture: generic_14 (../resources/textures/generics/generic_14.png)
Loaded texture: generic_16 (../resources/textures/generics/generic_16.png)
Loaded texture: generic_6 (../resources/textures/generics/generic_6.png)
Loaded texture: generic_17 (../resources/textures/generics/generic_17.png)


# TODO: support layers/groups for showing/hiding meshes in mgl window 

In [ ]:
py = ([vec3(0.9423, 0.9423, 0.0577), vec3(-0.9423, 0.9423, 0.0577), vec3(-0.9423, -0.9423, 0.0577), vec3(0.9423, -0.9423, 0.0577)], [[vec3(0.6000, 0.6000, 0.0577), vec3(0.4000, 0.6000, 0.0577), vec3(0.4000, -0.6000, 0.0577), vec3(0.6000, -0.6000, 0.0577)], [vec3(-0.6000, -0.6000, 0.0577), vec3(-0.4000, -0.6000, 0.0577), vec3(-0.4000, -0.4000, 0.0577), vec3(-0.6000, -0.4000, 0.0577)], [vec3(-0.4000, 0.6000, 0.0577), vec3(-0.6000, 0.6000, 0.0577), vec3(-0.6000, 0.4000, 0.0577), vec3(-0.4000, 0.4000, 0.0577)]])
t = triangulation(py, 0.0001, None)

f, ax = plot()
plot_polygon(ax, py, lw=6)
for tri in t.simplices():
    plot_loop(ax, list(tri), col='g')


In [ ]:
def edges_to_loops(edges):
    loops = [list(edges[0])]
    unfin = edges[1:]
    while unfin:
        u, v = unfin.pop(0)
        w = None
        for loop in loops:
            for i, p in enumerate(loop):
                if p.isnear(u):
                    w = v
                    break
                elif p.isnear(v):
                    w = u
                    break
            if w is not None:
                if i > 0:
                    loop.append(w)
                else:
                    loop.insert(0, w)
                break
                    
                    
    return loops

def loops_to_edges(loops):
    edges = []
    return edges